In [2]:
# !pip install tensorflow

In [35]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import os

In [36]:
# print(tf.__version__)

image_width = 224
image_height = 224 

dataset_path = "/Users/aguszulvani/privacy/MKOM/TESIS/CODE/dataset"

# Load training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    validation_split=0.2,  # 20% data for validation
    subset="training",
    seed=123,  # Random seed for reproducibility
    # image_size=(image_width, image_height),  # Resize all images to a fixed size
    batch_size=32  # Number of images per batch
)

# Load validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    validation_split=0.2,
    subset="validation",
    seed=123,
    # image_size=(image_width, image_height),
    batch_size=32
)

# Get class names before further processing
class_names = train_ds.class_names
print("Class names:", class_names)


Found 241 files belonging to 3 classes.
Using 193 files for training.
Found 241 files belonging to 3 classes.
Using 48 files for validation.
Class names: ['button', 'input-text', 'label']


In [29]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [30]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [31]:
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)


Image batch shape: (32, 256, 256, 3)
Label batch shape: (32,)


2024-12-07 20:03:25.554975: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [40]:
class_names = sorted(os.listdir(dataset_path))
print("Class names:", class_names)

# Define a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_ds.class_names), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=32
)


Class names: ['.DS_Store', 'button', 'input-text', 'label']
Epoch 1/32


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d_26" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 256, 256, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 256, 256, 3), dtype=float32)
  • training=True
  • mask=None

In [34]:
# Save the model to a file
model.save("cnn_model.keras")


NameError: name 'model' is not defined

In [25]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("cnn_model.keras")


In [45]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the image
image_path = "/Users/aguszulvani/privacy/MKOM/TESIS/CODE/data-test/label.png"
image = Image.open(image_path).convert("RGB")

# Resize the image to match the model's input size (224x224)
image = image.resize((224, 224))

# Convert the image to a NumPy array
image_array = np.array(image)

# Normalize pixel values to the range [0, 1]
image_array = image_array / 255.0

# Add a batch dimension (model expects input shape [batch_size, height, width, channels])
image_array = np.expand_dims(image_array, axis=0)


In [46]:
# Predict the class probabilities
predictions = model.predict(image_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted class: button
